In [1]:
from rdflib import Graph

from viscars.dao import ContentRecommenderDAO, VisualizationRecommenderDAO
from viscars.recommenders.cacf import ContextAwareCollaborativeFiltering

## Load the data

In [2]:
graph_ = Graph()
graph_.parse('../data/protego/protego_ddashboard.ttl')
graph_.parse('../data/protego/protego_zplus.ttl')
graph_.parse('../data/protego/visualizations.ttl')

<Graph identifier=N9cdf743b922d4e02a837ccd9ab4182c5 (<class 'rdflib.graph.Graph'>)>

## Build the two-stage recommender system

In [3]:
class RecommendationPipeline:

    def __init__(self):
        self.content_dao = ContentRecommenderDAO(graph_)
        self.content_recommender = ContextAwareCollaborativeFiltering(self.content_dao, cbcf_w=0.5, ubcf_w=0.5, verbose=False)
        self.vis_dao = VisualizationRecommenderDAO(graph_)
        self.visualization_recommender = ContextAwareCollaborativeFiltering(self.vis_dao, ubcf_w=1, verbose=False)

    def recommend_dashboard(self, u_id, c_id):
        content_recommendations = self.content_recommender.predict(u_id, c_id, k=5)

        # Find cutoff for Multiple-View recommendation
        # We recommend the top x items, where x is the average number of items rated by users in the context
        ratings = self.content_dao.ratings[(self.content_dao.ratings['c_id'] == c_id)]
        c = int(ratings.value_counts('u_id').mean())

        dashboard = []
        for r in content_recommendations[:c]:
            visualization_recommendations = self.visualization_recommender.predict(u_id, r['itemId'], k=5)
            recommendation = visualization_recommendations[0]
            dashboard.append({'propertyId': recommendation['contextId'], 'visualizationId': recommendation['itemId']})
        return dashboard

## Generate dashboards examples

In [4]:
recommender = RecommendationPipeline()

[2024-02-12 16:59:37,209] - [viscars.recommenders.base] - [INFO] Building model...
[2024-02-12 16:59:37,210] - [viscars.recommenders.base] - [INFO] Calculating CBCF similarities...
[2024-02-12 17:00:19,695] - [viscars.recommenders.base] - [INFO] Calculating UBCF similarities...
[2024-02-12 17:00:19,709] - [viscars.recommenders.base] - [INFO] Model built.
[2024-02-12 17:00:21,062] - [viscars.recommenders.base] - [INFO] Building model...
[2024-02-12 17:00:21,062] - [viscars.recommenders.base] - [INFO] Building model...
[2024-02-12 17:00:21,063] - [viscars.recommenders.base] - [INFO] Calculating UBCF similarities...
[2024-02-12 17:00:21,063] - [viscars.recommenders.base] - [INFO] Calculating UBCF similarities...
[2024-02-12 17:00:21,072] - [viscars.recommenders.base] - [INFO] Model built.
[2024-02-12 17:00:21,072] - [viscars.recommenders.base] - [INFO] Model built.


### Patient (Diabetes) - Nurse

In [5]:
user = 'https://dynamicdashboard.ilabt.imec.be/users/5'  # Nurse
context = 'http://example.com/tx/patients/zplus_6'  # Diabetes

nurse_diabetes = recommender.recommend_dashboard(user, context)

In [6]:
print(nurse_diabetes)

[{'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/enriched-call'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%3A77%3A71%3A7D%3A93%3AD7%2Fservice0009/properties/org.dyamand.types.health.GlucoseLevel', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.AQURA_10_10_145_9/properties/org.dyamand.aqura.AquraLocationState_Protego%20User', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/scrolling-table'}]


### Patient (HeartDisease) - Nurse

In [7]:
user = 'https://dynamicdashboard.ilabt.imec.be/users/5'  # Nurse
context = 'http://example.com/tx/patients/zplus_235'  # HeartDisease

nurse_heartdisease = recommender.recommend_dashboard(user, context)

In [8]:
print(nurse_heartdisease)

[{'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.lifestyle/properties/enriched-call', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/enriched-call'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A9D%3A6B%3A89%3A43%3ACD%2Fservice0025/properties/org.dyamand.types.health.DiastolicBloodPressure', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A9D%3A6B%3A89%3A43%3ACD%2Fservice0025/properties/org.dyamand.types.health.SystolicBloodPressure', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A1C%3A05%3AFF%3AA9%3A4E%2Fservice0029/propertie

### Patient (HeartDisease) - Operator

In [9]:
user = 'https://dynamicdashboard.ilabt.imec.be/users/4'  # Operator
context = 'http://example.com/tx/patients/zplus_235'  # HeartDisease

operator_heartdisease = recommender.recommend_dashboard(user, context)

In [10]:
print(operator_heartdisease)

[{'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.lifestyle/properties/enriched-call', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/enriched-call'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A9D%3A6B%3A89%3A43%3ACD%2Fservice0025/properties/org.dyamand.types.health.DiastolicBloodPressure', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A9D%3A6B%3A89%3A43%3ACD%2Fservice0025/properties/org.dyamand.types.health.SystolicBloodPressure', 'visualizationId': 'http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'}, {'propertyId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_235.00%3A1C%3A05%3AFF%3AA9%3A4E%2Fservice0029/propertie